# DHT22 Prediction Model

The training code is provided by [@namyh18](https://github.com/namyh18).

In [ ]:
import json
import torch
import torch.nn as nn
import torch.onnx
import numpy as np
from torch.utils.data import DataLoader, Dataset

### Class Definitions

In [ ]:
class DHTDataset(Dataset):
    def __init__(self, key):
        self.data = []
        self.target = []
        self.key = key

        db = open("db.json", "r")
        humi_temp = json.load(db)

        for i in range(len(humi_temp.get(self.key))-62):
            tmp_data = []
            tmp_data.append([float(humi_temp.get(self.key)[i][1])])
            tmp_data.append([float(humi_temp.get(self.key)[i+1][1])])
            tmp_data.append([float(humi_temp.get(self.key)[i+2][1])])
            self.data.append(torch.as_tensor(tmp_data))

            self.target.append(
                torch.as_tensor([float(humi_temp.get(self.key)[i+62][1])]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data, target = self.data[index], self.target[index]
        return data, target

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()

        self.linear = nn.Sequential(
            nn.Linear(in_features=1, out_features=16, bias=True),
            nn.ReLU(inplace=False),
            nn.Linear(in_features=16, out_features=32, bias=True),
            nn.ReLU(inplace=False)
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=96, out_features=32, bias=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=32, out_features=1, bias=True)
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

### Function Definitions

##### A function to split dataset into several data loaders

In [ ]:
def splitDataset(dataset):
    test_size = int(len(dataset)/6)
    val_size = int(len(dataset)/6)
    train_size = len(dataset) - test_size - val_size

    trainset, valset, testset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size])

    trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
    validloader = DataLoader(valset, batch_size=16, shuffle=True)
    testloader = DataLoader(testset, batch_size=16, shuffle=True)

    return trainloader, validloader, testloader

##### A function to train model

In [ ]:
def train_model(model, patience, num_epochs, train_loader, valid_loader):
    train_losses = []
    valid_losses = []
    mean_train_losses = []
    mean_valid_losses = []
    p = 0
    min_valid_loss = float("inf")

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1, num_epochs + 1):

        model.train()
        for train_data, train_target in train_loader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                train_data = train_data.to(device, dtype=torch.float)
                train_target = train_target.to(device, dtype=torch.float)
            optimizer.zero_grad()
            output = model(train_data)
            loss = criterion(output, train_target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()

        for valid_data, valid_target in valid_loader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                valid_data = valid_data.to(device, dtype=torch.float)
                valid_target = valid_target.to(device, dtype=torch.float)
            output = model(valid_data)
            loss = criterion(output, valid_target)
            valid_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        valid_loss = np.mean(valid_losses)
        mean_train_losses.append(train_loss)
        mean_valid_losses.append(valid_loss)

        if min_valid_loss > valid_loss:
            min_valid_loss = valid_loss
            print(f"min_valid_loss: {min_valid_loss}")

        epoch_len = len(str(num_epochs))
        print_msg = (f"[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] " +
                     f"train_loss: {train_loss:.5f} " +
                     f"valid_loss: {valid_loss:.5f} ")
        print(print_msg)

        train_losses = []
        valid_losses = []

        if min_valid_loss < valid_loss and epoch > 1:
            p = p + 1
            print(f'patience: {p}')
        else:
            p = 0
            torch.save(model.state_dict(), "bestmodel.pt")
            print("Saving Model...")

        if patience == p:
            print("Early stopping")
            break

    model.load_state_dict(torch.load("bestmodel.pt"))

    return model

### Training

##### Load datasets and get dataloaders

In [ ]:
tempDataset = DHTDataset("temperature")
humiDataset = DHTDataset("humidity")

temp_trainloader, temp_validloader, temp_testloader = splitDataset(tempDataset)
humi_trainloader, humi_validloader, humi_testloader = splitDataset(humiDataset)

##### Make models

In [ ]:
tempModel = SimpleModel()
humiModel = SimpleModel()

if torch.cuda.is_available():
    tempModel = tempModel.cuda()
    humiModel = humiModel.cuda()

##### Do training with earlystopping

In [ ]:
temp_predict_model = train_model(tempModel, patience=30, num_epochs=1000,
                                 train_loader=temp_trainloader,
                                 valid_loader=temp_validloader)
torch.save(temp_predict_model.state_dict(), "temp_predict.pt")

In [ ]:
humi_predict_model = train_model(humiModel, patience=30, num_epochs=1000,
                                 train_loader=humi_trainloader,
                                 valid_loader=humi_validloader)
torch.save(humi_predict_model.state_dict(), "humi_predict.pt")

### ONNX Coversion

In [ ]:
def convert_to_onnx(saved_file_name):
    saved_model = SimpleModel()
    saved_model.load_state_dict(torch.load(saved_file_name))
    saved_model.eval()
    torch.onnx.export(
        saved_model,
        torch.randn((1, 3, 1)),
        saved_file_name[:-3] + '.onnx',
        opset_version=11,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"]
    )

In [ ]:
convert_to_onnx("temp_predict.pt")
convert_to_onnx("humi_predict.pt")

# convert_to_onnx("example/inference/temp_predict.pt")
# convert_to_onnx("example/inference/humi_predict.pt")

In [ ]:
# End of Document.